# Reconocimiento de señales de tráfico
### Máster Universitario en Ingeniería computacional y matemática - Área de inteligencia artificial
### Antonio González Hidalgo (agonzalezhidalgo@uoc.edu)

Usando una red neuronal convolucional.

Para la correcta funcionamiento de este notebook, el notebook debe de estar estructurado de la siguiente manera:

- ./dataset
- ./dataset/info.csv          Fichero que contiene los nombres de las señales correspondientes.
- ./dataset/train/Images      Conjunto de imágenes que formarán el entrenamiento de la red neuronal.
- ./dataset/test/info.csv.    Fichero que contiene las categórias de las imágenes de test.
- ./dataset/test/Images       Conjunto de imágenes que constituirán el test.

In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (64, 64)

# Configuramos la CNN
EPOCHS = 15
OPTIMIZERS = ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']
ACTIVATION = 'elu'

print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
train_path = os.path.join(ROOT_PATH, "dataset_det/train")
test_path = os.path.join(ROOT_PATH, "dataset_det/test")

Tamaño de las imágenes de entrada:  (64, 64)
Vectorizando la entrada, sería de un tamaño:  4096


In [3]:
# Cargamos las imágenes de entrenamiento y de test.
images_train, labels_train = traffic.readDataset(train_path, IMG_SHAPE, False)
images_test, labels_test = traffic.readDataset(test_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)


# Se imprime información de los datos cargados.
traffic.print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
traffic.print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")

Total images (train):  15481
Total labels (train):  2
Images shape:  (15481, 64, 64, 3)
Labels shape:  (15481,)
Total images (test):  14726
Total labels (test):  2
Images shape:  (14726, 64, 64, 3)
Labels shape:  (14726,)


In [4]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [1. 0.]


In [5]:
def get_keras_model(activation):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation=activation, input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model

In [6]:
for optimizer in OPTIMIZERS:
    print(optimizer)
    model = get_keras_model(ACTIVATION)
    model.compile(loss="categorical_crossentropy",
                 optimizer=optimizer,
                 metrics=['accuracy'])

    model.fit(np_images_train, labels_categorical_train, verbose = 0,
              batch_size = 32,
              epochs = EPOCHS,
              callbacks=[ModelCheckpoint('model_64_64_det_' + optimizer +'.h5', save_best_only = False)])

    # Evaluación del modelo
    test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
    print('Test loss:', test_loss)
    print('Test accuracy:', test_acc)

sgd
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
14726/14726 [==============================] - 27s 2ms/step
Test loss: 0.6680826799910904
Test accuracy: 0.7391009099470861
rmsprop
14726/14726 [==============================] - 26s 2ms/step
Test loss: 1.8174682603340269
Test accuracy: 0.8429987776721445
adagrad
14726/14726 [==============================] - 42s 3ms/step
Test loss: 0.599584840472348
Test accuracy: 0.7824935488252072
adadelta
14726/14726 [==============================] - 44s 3ms/step
Test loss: 1.2000004304088578
Test accuracy: 0.8020507945131061
adam
14726/14726 [==============================] - 44s 3ms/step
Test loss: 0.7956833449665316
Test accuracy: 0.9049979627869075
adamax
14726/14726 [==============================] - 46s 3ms/step
Test loss: 0.9233878176119974
Test accuracy: 